<a href="https://colab.research.google.com/github/sauravdas093/Mercor/blob/main/Mercor_Clothing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Mounting Google Drive to access csv files in Colab.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#Importing the pandas library as pd alias
import pandas as pd    

# Reading the database of the clothing items from multiple e-commerce websites
flipkart = pd.read_csv('/content/drive/MyDrive/Mercor/flipkart.csv')
meesho = pd.read_csv('/content/drive/MyDrive/Mercor/meesho.csv')
myntra = pd.read_csv('/content/drive/MyDrive/Mercor/myntra.csv')

In [2]:
#Concat all the data from various e-commerce websites
clothing_df = pd.concat([flipkart, meesho, myntra],ignore_index=True)
clothing_df = clothing_df.reset_index()

#Alloting column names
clothing_df.columns = ['Id','Image_url', 'Brand', 'Product_url','Product','Image','product_url_nan']
clothing_df.drop(['Image','product_url_nan'],axis=1,inplace= True)
clothing_df.dropna(inplace=True)
result_df = clothing_df[['Id']].copy()
print(clothing_df.shape)
clothing_df.head(5)

(20101, 5)


,Id,Image_url,Brand,Product_url,Product
0,0,https://rukminim1.flixcart.com/image/612/612/x...,SAN TEE,https://www.flipkart.com/san-tee-track-pant-bo...,Track Pant For Boys & Girls
1,1,https://rukminim1.flixcart.com/image/612/612/x...,SILVER TEE,https://www.flipkart.com/silver-tee-short-boys...,Short For Boys Casual Printed Cotton Blend
2,2,https://rukminim1.flixcart.com/image/612/612/x...,Mars Infiniti,https://www.flipkart.com/mars-infiniti-boys-ca...,Boys Casual T-shirt Shorts
3,3,https://rukminim1.flixcart.com/image/612/612/x...,Fasla,https://www.flipkart.com/fasla-short-girls-cas...,Short For Girls Casual Printed Cotton Linen Blend
4,4,https://rukminim1.flixcart.com/image/612/612/x...,CUTE TREND,https://www.flipkart.com/cute-trend-short-boys...,Short For Boys Sports Printed Pure Cotton


In [3]:
#Import the nltk module
import nltk

#Import the WordNetLemmatizer class from the nltk.stem module
from nltk.stem import WordNetLemmatizer

#Create an instance of the WordNetLemmatizer class
wnl = WordNetLemmatizer()

#Download the WordNet dataset
nltk.download('wordnet')

#defining a function to lemmatize a text
def lemmatize(word_to_lemmatize):
  return(wnl.lemmatize(word_to_lemmatize))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#Preprocessing data and Text Normalization to remove special characters, lowercase conversion and lemmatization

clothing_df['Product'] = clothing_df['Product'].replace('\W', ' ', regex=True)
clothing_df['Product'] = clothing_df['Product'].str.lower()
clothing_df['Product'] = clothing_df['Product'].apply(lemmatize)

In [5]:
# Importing the Word2Vec model from the gensim library
from gensim.models import Word2Vec

# Importing the word_tokenize function from the nltk.tokenize module
from nltk.tokenize import word_tokenize

text_vector_mapping = {}

# Preprocess text (example)
# text1 = "sdfdsf sfgfdsgfd sfdgdsg pant for boys."

#Function to extract useful features from the text 
def word_embedding(input_text):
    text2 = input_text
    # tokens1 = word_tokenize(text1.lower())  # Tokenize and convert to lowercase
    tokens2 = word_tokenize(text2.lower())  # Tokenize and convert to lowercase

    # Train Word2Vec model (example)
    model = Word2Vec(sentences=[tokens2], min_count=1, vector_size=100)

    # Extract features using average word vectors
    def extract_features(text):
        tokens = word_tokenize(text.lower())  # Preprocess text
        vectors = [model.wv[word] for word in tokens if word in model.wv]  # Get word vectors
        if vectors:
            avg_vector = sum(vectors) / len(vectors)  # Calculate average vector
            return avg_vector
        else:
            return None

    # Convert vector back to text
    def vector_to_text(vector):
        text = text_vector_mapping.get(tuple(vector), "Vector representation not found.")
        return text

    # Example usage
    # features1 = extract_features(text1)
    features2 = extract_features(text2)

    # text_vector_mapping[tuple(features1)] = text1
    text_vector_mapping[tuple(features2)] = text2

    #Storing the final extracted text in extracted_text
    extracted_text = vector_to_text(features2)

    return extracted_text

In [6]:
#Import the dot function from the NumPy library
from numpy import dot

#Import the norm function from the numpy.linalg module
from numpy.linalg import norm

#Import the quote function from the urllib.parse module
from urllib.parse import quote

#Set the maximum column width for Pandas DataFrame display
pd.set_option('display.max_colwidth', 1000)

#creating resulting dataframe
result_df = clothing_df[['Id']].copy()

"""
Defining a function that takes input text data,
computes the similarity between the input text and the item descripotion
of each item in the database and returns a dataframe of cosine values of each item 
against the input text.
"""
def compute_similarity1(input_value):
  
  #Defining two empty lists
  cos_sim_values = []
  Product_url = []
  
  #Run a loop to compute similarity between two vectors
  for i in range(0,len(clothing_df)):
      
      #storing elements from the cell in Product_name
      Product_name = (clothing_df['Product'][i:i+1])

      #converting to string value
      Product_name = Product_name.to_string()

      # Removing any digit association with the text
      Product_name = ''.join([j for j in Product_name if not j.isdigit()])

      #storing elements from the cell in Product_url_clean
      Product_url_clean = (clothing_df['Product_url'][i:i+1])

      #converting to string value
      Product_url_clean = Product_url_clean.to_string()

      # Removing any digit association with the text
      Product_url_clean = ''.join([m for m in Product_url_clean if not m.isdigit()])
      
      #splitting the words and storing it in dataframe
      Product_name_df = pd.DataFrame({"Words": Product_name.split()})

      #lookup for the frequency of the words
      Product_name_df_freq = pd.DataFrame(pd.crosstab(index=Product_name_df['Words'],columns='count'))

      #splitting the words and storing it in dataframe
      input_df = pd.DataFrame({"Words": input_value.split()})

      #lookup for the frequency of the words
      input_df_freq = pd.DataFrame(pd.crosstab(index=input_df['Words'],columns='count'))

      #concat the dfs
      dfs = [Product_name_df_freq, input_df_freq]
      all_words = pd.concat(dfs, axis=1)

      # Substituting the NAN with zeros
      all_words = all_words.fillna(0)
      all_words.columns = ["exist_data", "input_data"]

      #computing the cosine values
      cos_sim = dot(all_words["exist_data"], all_words["input_data"])/(norm(all_words["exist_data"])*norm(all_words["input_data"]))

      #append the values in the corresponding lists
      cos_sim_values.append(cos_sim)
      Product_url.append(Product_url_clean)

  #storing the above lists values in the newly created column        
  result_df['cos_sim_values'] = cos_sim_values
  result_df['Product_url'] = Product_url

  #Return the value
  return result_df

In [7]:
# This is a function to encode and clean the returned url after processing
def encode_and_clean_url(url):
 encoded_url = (quote(url, safe=':/?&='))
 cleaned_url = (encoded_url.replace("%20%20%20%20",""))
 return cleaned_url

In [8]:
# Importing the json module for working with JSON data

import json

def process_payload(json_payload):
    # Parse the JSON payload
    data = json.loads(json_payload)
    
    # Access the input text from the JSON payload
    input_text = data["input_text"]
    return input_text
# Example JSON payload
# json_payload = {"input_text": "panty and bra"}
json_payload = input("Enter your value in json format: ")

# Call the function with the JSON payload
input_text = process_payload(json_payload)
input_text


Enter your value in json format: {"input_text": "pants for boys"}


'pants for boys'

In [9]:
nltk.download('punkt')

#calling the function to extract useful features
extracted_text = word_embedding(input_text)

# calling the function to measure cosine similarity
result_df = compute_similarity1(extracted_text) 

#sorting the cosine values in order of highest values in the top
result_df = result_df.sort_values(by='cos_sim_values', ascending=False)

#Calling the function to encode and clean the url
result_df['encoded_and_cleaned_url'] = result_df['Product_url'].apply(encode_and_clean_url)

#Displaying the result after the above-mentioned operations
result_df.head(4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Id,cos_sim_values,Product_url,encoded_and_cleaned_url
66,66,0.666667,https://www.flipkart.com/thampa-brief-boys/p/itmfhcebpzwrqawv?pid=KBEFHZXCGXZNZ&lid=LSTKBEFHZXCGXZNZYOXNVL&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_WubAMsDSnsOQfuBxQhOKKCuLUmtUfIKbjrSgiBUNQd%FLeMBiKCIicODOPnFcGsu%BcEIUtpeg%D%D&ppt=browse&ppn=browse&ssid=klcwszxog,https://www.flipkart.com/thampa-brief-boys/p/itmfhcebpzwrqawv?pid=KBEFHZXCGXZNZ&lid=LSTKBEFHZXCGXZNZYOXNVL&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_WubAMsDSnsOQfuBxQhOKKCuLUmtUfIKbjrSgiBUNQd%25FLeMBiKCIicODOPnFcGsu%25BcEIUtpeg%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog
1627,1627,0.577350,https://www.flipkart.com/xo-track-pant-boys/p/itmfdwfqejrds?pid=KTKFVHZTPJVEMHDS&lid=LSTKTKFVHZTPJVEMHDSTBORU&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_VNfBeOkqkiXHuH%FK%Fc%FAnjxuatPgwDvuqolYZqG%Frlmtg%FaXjCoWUeuMRmSjLEtZFiKdjaetVA%D%D&ppt=browse&ppn=browse&ssid=klcwszxog,https://www.flipkart.com/xo-track-pant-boys/p/itmfdwfqejrds?pid=KTKFVHZTPJVEMHDS&lid=LSTKTKFVHZTPJVEMHDSTBORU&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_VNfBeOkqkiXHuH%25FK%25Fc%25FAnjxuatPgwDvuqolYZqG%25Frlmtg%25FaXjCoWUeuMRmSjLEtZFiKdjaetVA%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog
3301,3301,0.577350,https://www.flipkart.com/stag-zoe-track-pant-boys/p/itmfefe?pid=KTKGNRPDAYGHEWK&lid=LSTKTKGNRPDAYGHEWKSUM&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_xSnjREsKjBiNdVpzQxopttffYZfWXwaEI%Fs%FzehhlykvqCUigOnnhzN%BuzuPquEKlRfw%D%D&ppt=browse&ppn=browse&ssid=klcwszxog,https://www.flipkart.com/stag-zoe-track-pant-boys/p/itmfefe?pid=KTKGNRPDAYGHEWK&lid=LSTKTKGNRPDAYGHEWKSUM&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_xSnjREsKjBiNdVpzQxopttffYZfWXwaEI%25Fs%25FzehhlykvqCUigOnnhzN%25BuzuPquEKlRfw%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog
986,986,0.577350,https://www.flipkart.com/xo-track-pant-boys/p/itmfdwfqejrds?pid=KTKFVHZTPJVEMHDS&lid=LSTKTKFVHZTPJVEMHDSTBORU&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_NKrxXuxvXBnsXIZRKz%FSrI%FyNdg%FUqIDAXRKLwJWeCfIiPNfVmOANmMRmSjLEtZFiKdjaetVA%D%D&ppt=browse&ppn=browse&ssid=klcwszxog,https://www.flipkart.com/xo-track-pant-boys/p/itmfdwfqejrds?pid=KTKFVHZTPJVEMHDS&lid=LSTKTKFVHZTPJVEMHDSTBORU&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_NKrxXuxvXBnsXIZRKz%25FSrI%25FyNdg%25FUqIDAXRKLwJWeCfIiPNfVmOANmMRmSjLEtZFiKdjaetVA%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog


In [10]:
#Inserting the rank index to rank the output
result_df.insert(0, 'Rank_of_the_output', range(1, len(result_df) +1) ) 
#Dropping the Id column
result_df.drop(['Id'],axis=1,inplace= True)

In [11]:
#Creating a new dataframe taking only below columns from result_df and storing in final_output
final_output = result_df[['Rank_of_the_output','cos_sim_values','encoded_and_cleaned_url']]

#Defining a function to convert string into json format
def json_format(string):
  return(json.dumps(string))

final_output_json_format = final_output['encoded_and_cleaned_url'].apply(json_format)

#Displaying all the url in ranked conditions in json format with the most similar item in the top
final_output_json_format.head(5)

66                    "https://www.flipkart.com/thampa-brief-boys/p/itmfhcebpzwrqawv?pid=KBEFHZXCGXZNZ&lid=LSTKBEFHZXCGXZNZYOXNVL&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_WubAMsDSnsOQfuBxQhOKKCuLUmtUfIKbjrSgiBUNQd%25FLeMBiKCIicODOPnFcGsu%25BcEIUtpeg%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog"
1627    "https://www.flipkart.com/xo-track-pant-boys/p/itmfdwfqejrds?pid=KTKFVHZTPJVEMHDS&lid=LSTKTKFVHZTPJVEMHDSTBORU&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_VNfBeOkqkiXHuH%25FK%25Fc%25FAnjxuatPgwDvuqolYZqG%25Frlmtg%25FaXjCoWUeuMRmSjLEtZFiKdjaetVA%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog"
3301                    "https://www.flipkart.com/stag-zoe-track-pant-boys/p/itmfefe?pid=KTKGNRPDAYGHEWK&lid=LSTKTKGNRPDAYGHEWKSUM&marketplace=FLIPKART&store=clo&srno=b__&otracker=browse&fm=organic&iid=en_xSnjREsKjBiNdVpzQxopttffYZfWXwaEI%25Fs%25FzehhlykvqCUigOnnhzN%25BuzuPquEKlRfw%25D%25D&ppt=browse&ppn=browse&ssid=klcwszxog"
986          